In [176]:
from gurobipy import *
import numpy as np
import pandas as pd
from Hourbit import *
from Fibit import *
from blockbit import *


In [191]:
BS_bits = Blockbit()[0]
BD_bits = Blockbit()[1]
HS_bits = HourBit()[0]
HD_bits = HourBit()[1]
FS_bits = Fibit()[0]
FD_bits = Fibit()[1]

In [194]:
Blockbit()[1]

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,1110,1,2,B,694.0,599.67,20,NaN
1,1106,1,2,B,562.0,553.92,11,NaN
2,1105,1,15,B,347.0,609.00,4,NaN
3,1102,1,15,B,543.0,581.19,8,NaN
4,1109,1,20,B,808.0,685.34,4,NaN
...,...,...,...,...,...,...,...,...
80,1078,1,2,B,129.0,716.49,19,NaN
81,1199,1,3,B,850.0,444.24,21,NaN
82,1090,1,11,B,172.0,563.73,10,NaN
83,1098,1,6,B,921.0,648.55,19,NaN


In [178]:
L_D = len(BD_bits)
L_S = len(Blockbit()[0])

In [179]:
Delta_S = np.zeros([L_S,24])


Delta_D = np.zeros([L_D,24])
BD_bits  = Blockbit()[1]
for i in range (L_D):
    if BD_bits['Hour'][i]+BD_bits['Duration'][i]<=24:
        Delta_D[i,BD_bits['Hour'][i]-1:BD_bits['Hour'][i]+BD_bits['Duration'][i]] = 1

BS_bits = Blockbit()[0]
for i in range (L_S):
    if BS_bits['Hour'][i]+BS_bits['Duration'][i]<=24:
        Delta_S[i,BS_bits['Hour'][i]-1:BS_bits['Hour'][i]+BS_bits['Duration'][i]] = 1

In [180]:
E = Model("Energy")
E.params.NonConvex =2
E.update()
J = np.arange(1,find_largest(HIndex_D)+1)
K = np.arange(1,find_largest(HIndex_S)+1)
H = np.arange(1,25)

Set parameter NonConvex to value 2


In [181]:
xD = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xD')
xS = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xS')
w_D = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS,name ='W_D')
w_S = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS,name ='W_S')
yD = E.addVars(L_D,ub=1,lb=0,  vtype=GRB.BINARY,name ='yD')
yS = E.addVars(L_S,ub=1,lb=0,  vtype=GRB.BINARY,name ='yS')

In [182]:
Data_H_D=HourBit()[1]
Data_H_SP = HourBit()[0]
BD_bits

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,1110,1,2,B,694.0,599.67,20,NaN
1,1106,1,2,B,562.0,553.92,11,NaN
2,1105,1,15,B,347.0,609.00,4,NaN
3,1102,1,15,B,543.0,581.19,8,NaN
4,1109,1,20,B,808.0,685.34,4,NaN
...,...,...,...,...,...,...,...,...
80,1078,1,2,B,129.0,716.49,19,NaN
81,1199,1,3,B,850.0,444.24,21,NaN
82,1090,1,11,B,172.0,563.73,10,NaN
83,1098,1,6,B,921.0,648.55,19,NaN


In [183]:
E.setObjective(sum(0.5*(2*Pair(j,h,Data_H_D,'DP')[0]+xD[h,j]*(Pair(j,h,Data_H_D,'DP')[1]-Pair(j,h,Data_H_D,'DP')[0]))*xD[h,j]*(Pair(j,h,Data_H_D,'DQ')[1]-Pair(j,h,Data_H_D,'DQ')[0])for j in J for h in H)
                -sum(0.5*(2*Pair(j,h,Data_H_SP,'SP')[0]+xS[h,j]*(Pair(j,h,Data_H_SP,'SP')[1]-Pair(j,h,Data_H_SP,'SP')[0]))*xS[h,j]*(Pair(j,h,Data_H_SP,'SQ')[1]-Pair(j,h,Data_H_SP,'SQ')[0])for j in J for h in H)
                +sum(BD_bits['Duration'][i]*BD_bits['Quantity'][i]*BD_bits['price'][i]*yD[i] for i in range(L_D)) - sum(BS_bits['Duration'][i]*BS_bits['Quantity'][i]*BS_bits['price'][i]*yS[i] for i in range(L_S))
,GRB.MAXIMIZE)

In [184]:

for h in H:
        E.addConstr(-sum((Pair(j,h,Data_H_D,'DQ')[1]-Pair(j,h,Data_H_D,'DQ')[0]) * xD[h,j] for j in J ) +sum((Pair(j,h,Data_H_SP,'SQ')[1]-Pair(j,h,Data_H_SP,'SQ')[0])*xS[h,j] for j in J) -Pair(1,h,Data_H_D,'DQ')[0] +Pair(1,h,Data_H_SP,'SQ')[0] 
                    +sum(Delta_S[i,h-1]*BS_bits['Quantity'][i]*yS[i] for i in range(L_S))  -sum(Delta_D[i,h-1]*BD_bits['Quantity'][i]*yD[i] for i in range(L_D))
                    ==0) 

# Demand
E.addConstrs(w_D[h,1]<=xD[h,1] for h in H)
E.addConstrs(xD[h,1] <=1 for h in H)
E.addConstrs(w_D[h,j]<=xD[h,j] for j in range(2,find_largest(HIndex_D)+1) for h in H)
E.addConstrs(xD[h,j]<=w_D[h,j-1] for j in range(2,find_largest(HIndex_D)+1) for h in H)
# Supply
E.addConstrs(w_S[h,1]<=xS[h,1] for h in H)
E.addConstrs(xS[h,1] <=1 for h in H)
E.addConstrs(w_S[h,j]<=xS[h,j] for j in range(2,find_largest(HIndex_S)+1) for h in H)
E.addConstrs(xS[h,j]<=w_S[h,j-1] for j in range(2,find_largest(HIndex_S)+1) for h in H)


{(2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Update*>,
 (2, 11): <gurobi.Constr *Awaiting Model Update*>,
 (2, 12): <gurobi.Constr *Awaiting Model Update*>,
 (2, 13): <gurobi.Constr *Awaiting Model Update*>,
 (2, 14): <gurobi.Constr *Awaiting Model Update*>,
 (2, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 16): <gurobi.Constr *Awaiting Model Update*>,
 (2, 17): <gurobi.Constr *Awaiting Model Update*>,
 (2, 18): <gurobi.Constr *Awaiting Model Update*>,
 (2, 19): <gurobi.Constr *Awaiting Model Update*>,
 (2, 20): <gurobi.Constr *Awaiting Model

In [185]:
E.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 95928 rows, 96104 columns and 241915 nonzeros
Model fingerprint: 0xd142ea5e
Model has 47952 quadratic objective terms
Variable types: 95904 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [1e+02, 1e+07]
  QObjective range [2e-02, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Presolve removed 48000 rows and 47982 columns
Presolve time: 0.16s
Presolved: 47928 rows, 48122 columns, 145963 nonzeros
Presolved model has 47952 quadratic objective terms
Variable types: 47952 continuous, 170 integer (170 binary)

Root relaxation: objective -6.094199e+08, 36378 iterations, 4.67 seconds (35.95 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 

In [186]:
Res_D = []
Res_S = []
for i in range(1,1000):

    Res_D.append(E.getAttr('x',xD)[(2,i)])
    Res_S.append(E.getAttr('x',xS)[(2,i)])


for i in range(len(Res_D)):
    if Res_D[i] >= 1e-8 and Res_D[i] <= 0.999:
        print(i+1)
for i in range(len(Res_S)):
    if Res_S[i] >= 1e-8 and Res_S[i] <= 0.999:
        print(i+1)

303
303


In [187]:
# def delta(SD):
#     if  SD == 'D':
#         BD_bits  = Blockbit()[1]
#         for i in range (L_D):
#             if BD_bits['Hour'][i]+BD_bits['Duration'][i]<=24:
#                 Delta_D[i,BD_bits['Hour'][i]-1:BD_bits['Hour'][i]+BD_bits['Duration'][i]] = 1
#         return(Delta_D)
#     if  SD == 'S':
#         BS_bits = Blockbit()[0]
#         for i in range (L_S):
#             if BS_bits['Hour'][i]+BS_bits['Duration'][i]<=24:
#                 Delta_S[i,BS_bits['Hour'][i]-1:BS_bits['Hour'][i]+BS_bits['Duration'][i]] = 1
#         return(Delta_S)


BD_bits  = Blockbit()[1]
for i in range (L_D):
    if BD_bits['Hour'][i]+BD_bits['Duration'][i]<=24:
        Delta_D[i,BD_bits['Hour'][i]-1:BD_bits['Hour'][i]+BD_bits['Duration'][i]] = 1

BS_bits = Blockbit()[0]
for i in range (L_S):
    if BS_bits['Hour'][i]+BS_bits['Duration'][i]<=24:
        Delta_S[i,BS_bits['Hour'][i]-1:BS_bits['Hour'][i]+BS_bits['Duration'][i]] = 1


In [188]:
BD_bits  = Blockbit()[1]
BD_bits['Hour'][0]+BD_bits['Duration'][0]

22

In [189]:
Blockbit()[1]

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,1110,1,2,B,694.0,599.67,20,NaN
1,1106,1,2,B,562.0,553.92,11,NaN
2,1105,1,15,B,347.0,609.00,4,NaN
3,1102,1,15,B,543.0,581.19,8,NaN
4,1109,1,20,B,808.0,685.34,4,NaN
...,...,...,...,...,...,...,...,...
80,1078,1,2,B,129.0,716.49,19,NaN
81,1199,1,3,B,850.0,444.24,21,NaN
82,1090,1,11,B,172.0,563.73,10,NaN
83,1098,1,6,B,921.0,648.55,19,NaN


In [190]:
sum(Delta_S[i,h]*Blockbit()[0]['Quantiy'][i]*yS for i in range(L_S))-sum(Delta_D[i,h]*Blockbit()[1]['Quantiy'][i]*yD for i in range(L_D))


IndexError: index 24 is out of bounds for axis 1 with size 24

In [ ]:
Delta_D[0]